In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
durch = pd.DataFrame()
null = pd.DataFrame()
durch['2mm'] = np.genfromtxt('data/VI_2mm.txt')
durch['6mm'] = np.genfromtxt('data/IV_60_Alu.txt')
durch['8mm'] = np.genfromtxt('data/VI_8mm.txt')
durch['15mm'] = np.genfromtxt('data/VI_15mm.txt')
durch['20mm'] = np.genfromtxt('data/VI_20mm.txt')
null['Null'] = np.genfromtxt('data/IV_60_N0.txt')

In [ ]:
# Zählrate bestimmen
durch_rate = pd.DataFrame()
for i in durch.columns:
    durch_rate[i] = durch[i].iloc[2:] / (durch[i].iloc[0]-durch[i].iloc[1])
null_rate = pd.DataFrame()
null_rate['Null'] = null['Null'].iloc[2:] / (null['Null'].iloc[0]-null['Null'].iloc[1])

In [ ]:
# Nullrate abziehen
durch_cleaned = pd.DataFrame()
for i in durch_rate.columns:
    durch_cleaned[i] = durch_rate[i] - null_rate['Null']

In [ ]:
delta_null = np.sqrt(null['Null'].iloc[2:].sum())/ (null['Null'].iloc[0]-null['Null'].iloc[1])
durch_n_delta = []
for i in durch.columns:
    durch_n_delta.append(np.sqrt(durch[i].iloc[2:].sum())/ (durch[i].iloc[0]-durch[i].iloc[1]))
delta_n = np.sqrt(np.asarray(durch_n_delta)**2 + delta_null**2)

In [ ]:
from scipy.optimize import curve_fit
def exp_fit(x, a, b):
    return a*(1 - np.exp(b*x))

In [ ]:
durchmesser_summe = durch_cleaned.sum()
d = [2, 6, 8, 15, 20]
plt.errorbar(d, durchmesser_summe, xerr=0.05, yerr=delta_n, fmt='x', label='$\dot{N}_G - \dot{N}_0$')
plt.xlabel('Durchmesser Streukörper [mm]')
plt.ylabel('Gesamtzählrate [s⁻¹]')
fit, std = curve_fit(exp_fit, d, durchmesser_summe, p0=[20, -1])
plt.plot(np.linspace(2, 20, 100), [ exp_fit(i, fit[0], fit[1]) for i in np.linspace(2,20,100)], label='Fit')
plt.legend()
plt.savefig('images/durchmesser.png', dpi=300)